# Chapter3 When Objects are Alike

Covering:
1. Inheritance
2. Extending built-in classes

A **superclass**, or parent class is a class that is being inherited from. A **subclass** is a class that is inheriting / derived / from or extends the superclass. In the following case, `object` **superclass** and `MySubClass` is the **subclass**.

In [1]:
class MySubClass(object):
    pass

The most obvious use of inheritance is to add functionality to an existing class.

In [2]:
# track name and e-mail for our contacts
class Contact(object):

    all_contacts = []
    
    def __init__( self, name, email ):
        self.name = name
        self.email = email
        # whenever we created a new contact, it will
        # be added to the list
        Contact.all_contacts.append(self)

`all_contacts` is a **class variables**. This variable is shared by all instances of this class meaning that there is only one `Contact.all_contacts list`. Note that we called `self.all_contacts`, this will then create a new instance variable on the object.

In [3]:
# inherit from Contact class, but has an additional method order
class Supplier(Contact):
    def order( self, order ):
        print( "send {} order to {}".format( order, self.name ) )

In [4]:
c = Contact(  name = "Some Body", email = "somebody@example.net" )
s = Supplier( name = "Sup Plier", email = "supplier@example.net" )
print(c.name, c.email, s.name, s.email)
print(c.all_contacts)

Some Body somebody@example.net Sup Plier supplier@example.net
[<__main__.Contact object at 0x10717e2b0>, <__main__.Supplier object at 0x10717e278>]


In [5]:
s.order("I need pliers")

send I need pliers order to Sup Plier


The `Supplier` can do everything the `Contact` can do and all the other additional methods that it needs.

## Extending Built-In

If we want to search the contact list by name, instead by adding a method to the `Contact` class, we can add the method on the list itself.

In [6]:
# [] is equivalent to list()
class ContactList(list):
    def search( self, name ):
        matching_contacts = []
        for contact in self:
            if name in contact.name:
                matching_contacts.append(contact)
        return matching_contacts

class Contact1(object):
    all_contacts = ContactList()
    
    def __init__( self, name, email ):
        self.name = name
        self.email = email
        Contact1.all_contacts.append(self)

In [7]:
c1 = Contact1("John A", "johna@example.net")
c2 = Contact1("John B", "johnb@example.net")
c3 = Contact1("Jenna C", "jennac@example.net")
c1.all_contacts

In [8]:
[ c.name for c in Contact1.all_contacts.search('John') ]

['John A', 'John B']

Most built-in types can be similarly extended. Commonly extended built-ins are object, list, set, dict, file, and str. Numerical types such as int and float are also occasionally inherited from. Another example extending dicts.

In [9]:
class LongNameDict(dict):
    def longest_key(self):
        longest = None
        for key in self:
            if not longest or len(key) > len(longest):
                longest = key
        return longest

longkeys = LongNameDict()
longkeys['hello'] = 1
longkeys['longest yet'] = 5
longkeys['hello2'] = 'world'
longkeys.longest_key() 

'longest yet'

## Overriding and super

Our `Contact` class allows only a name and an e-mail, if we want to have a third attribute during the initialization, we can override `__init__`.

In [10]:
class Friend(Contact):
    def __init__( self, name, email, phone ):
        self.name  = name
        self.email = email
        self.phone = phone

But doing it this way makes it hard to maintain as our `Contact` and `Friend` class have duplicated code to set up the `name` and `email` property. Thus we can change the above to using the `super()` function, which returns the object as an instance of the parent class.

In [11]:
class Friend(Contact):
    def __init__( self, name, email, phone ):
        # We can use super() in any other places we want instead of just in __init__
        super().__init__( name, email )
        self.phone = phone

## Multiple Inheritance

Use cases: a superclass that is not meant to exist on its own, but is meant to be inherited by some other class to provide extra functionality.

In [12]:
class MailSender(object):
    def send_mail( self, message ):
        print( 'Sending mail to ' + self.email )

class EmailableContact( Contact1, MailSender ):
    pass

In [13]:
e = EmailableContact("John Smith", "jsmith@example.net")
print( e.send_mail("Hello, test e-mail here") )

Sending mail to jsmith@example.net
None


Another example. We define a `AddressHolder` to hold addresses and we can later use it to create other entities such as companies and buildings.

In [14]:
class AddressHolder(object):
    def __init__( self, street, city, state, code ):
        self.street = street
        self.city   = city
        self.state  = state
        self.code   = code

Using this with the `Friend` class. Since we're using multiple inheritance, an naive approach would be to directly call the `__init__` function of each superclass.

In [15]:
class Friend( Contact1, AddressHolder ):
    def __init__( self, name, email, phone,
                  street, city, state, code ):
        
        Contact.__init__( self, name, email )
        AddressHolder.__init__( self, street, city, state, code )
        self.phone = phone

The problem with this is the **diamond problem.** The `__init__` method from the `Friend` class first calls `__init__` on `Contact` which implicitly initializes the object superclass (remember, all classes derive from object). Friend then calls `__init__` on `AddressHolder`, which implicitly initializes the object superclass... again. The parent class has been set up twice. In this case, that's relatively harmless, but in some situations, it could spell disaster. Imagine trying to connect to a database twice for every request!

We can illustrate the behavior using the following example.

In [16]:
class BaseClass(object):
    num_base_calls = 0
    def call_me(self):
        print("Calling method on Base Class")
        self.num_base_calls += 1

class LeftSubclass(BaseClass):
    num_left_calls = 0
    def call_me(self):
        BaseClass.call_me(self)
        print("Calling method on Left Subclass")
        self.num_left_calls += 1

class RightSubclass(BaseClass):
    num_right_calls = 0
    def call_me(self):
        BaseClass.call_me(self)
        print("Calling method on Right Subclass")
        self.num_right_calls += 1

class Subclass(LeftSubclass, RightSubclass):
    num_sub_calls = 0
    def call_me(self):
        LeftSubclass.call_me(self)
        RightSubclass.call_me(self) 
        print("Calling method on Subclass") 
        self.num_sub_calls += 1

In [17]:
s = Subclass()
s.call_me()

Calling method on Base Class
Calling method on Left Subclass
Calling method on Base Class
Calling method on Right Subclass
Calling method on Subclass


In [18]:
print(s.num_sub_calls, s.num_left_calls, s.num_right_calls, s.num_base_calls)

1 1 1 2


We can see that the base class's `call_me` method has been called twice. Changing to use `super()`.

In [19]:
class BaseClass1(object):
    num_base_calls = 0
    def call_me(self):
        print("Calling method on Base Class")
        self.num_base_calls += 1
    
class LeftSubclass1(BaseClass1):
    num_left_calls = 0
    def call_me(self):
        super().call_me()
        print("Calling method on Left Subclass")
        self.num_left_calls += 1

class RightSubclass1(BaseClass1):
    num_right_calls = 0
    def call_me(self):
        super().call_me()
        print("Calling method on Right Subclass")
        self.num_right_calls += 1

class Subclass1( LeftSubclass1, RightSubclass1 ):
    num_sub_calls = 0
    def call_me(self):
        super().call_me()
        print("Calling method on Subclass")
        self.num_sub_calls += 1

In [20]:
s = Subclass1()
s.call_me()

Calling method on Base Class
Calling method on Right Subclass
Calling method on Left Subclass
Calling method on Subclass


In [21]:
print(s.num_sub_calls, s.num_left_calls, s.num_right_calls, s.num_base_calls)

1 1 1 1


Each hierarchy is executed once. First `call_me` of `Subclass` calls `super().call_me()`, which happens to refer to `LeftSubclass.call_me()`. `LeftSubclass.call_me()` then calls `super().call_ me()`, but in this case, `super()` is referring to `RightSubclass.call_me()`. This is the next method, not the parent method.

Thus for the `Friend` class, this will have to become the following ...................

In [22]:
class Contact2(object):
    all_contacts = []
    def __init__( self, name = '', email = '', **kwargs ): 
        super().__init__(**kwargs)
        self.name = name
        self.email = email 
        self.all_contacts.append(self)

class AddressHolder(object):
    def __init__( self, street = '', city = '', state = '', code = '', **kwargs ):
        super().__init__(**kwargs)
        self.street = street
        self.city = city
        self.state = state
        self.code = code

class Friend( Contact2, AddressHolder ):
    def __init__( self, phone = '', **kwargs ): 
        super().__init__(**kwargs) 
        self.phone = phone

**TakeAway: Multiple inheritance can be messy!!**

## Polymorphism

Different behaviors happen depending on which subclass is being used.

In [23]:
class AudioFile(object):
    
    def __init__( self, filename ):
        if not filename.endswith(self.ext): # access the ext variable in the subclass
            raise Exception('invalid file format')
        
        self.filename = filename
    
class MP3File(AudioFile):
    ext = 'mp3'   
    def play(self):
        print('playing {} as mp3'.format(self.filename) )
        
class WavFile(AudioFile):
    ext = "wav"
    def play(self):
        print("playing {} as wav".format(self.filename))

class OggFile(AudioFile):
    ext = "ogg"
    def play(self):
        print("playing {} as ogg".format(self.filename))

In [26]:
mp3 = MP3File("myfile.mp3")
# not_an_mp3 = MP3File("myfile.ogg")